In [358]:
import numpy as np
import matplotlib.pyplot as plt
from random import *
from math import *

<h3>##TODO: I need some feedback on efficiency, code beauty, and algorithm sanity here. Hit me up if you found something worthy of noting!</h3>

In [359]:
## profile_vector[0]: academic_score
## profile_vector[1]: sports_score
## profile_vector[2]: arts_score
## profile_vector[3]: race(0 for male and 1 for female)
## profile_vector[4]: grade level(1, 2, 3, 4 for years in high school)

def second_largest(array):
    # This is O(n) btw! Sorting and then indexing the second one would've been O(nlog(n))
    # But it's mutative so meh
    _lst = list(array)[:]
    _lst.remove(max(_lst))
    return max(_lst)


class Participant:
    def __init__(self, profile_vector):
        self.profile_vector = profile_vector
        self.interests = profile_vector[:3]
        self.background = profile_vector[3:5]
    
    def similarity(self, other, mode='dot_product'):
        assert mode in ['dot_product', 'gaussian_kernel'], 'Invalid mode "{0}"'.format(mode)
        if mode == 'dot_product':
            normalized_self = self.interests / np.linalg.norm(self.interests) 
            normalized_other = other.interests / np.linalg.norm(other.interests) 
            return np.float32(normalized_self.dot(normalized_other))
        elif mode == 'gaussian_kernel':
            sigma = 1 ## I have no idea what to set this to. Lmk if you know
            return e ** ((-np.linalg.norm(self.interests - other.interests) ** 2) / (2 * sigma ** 2))
        
    @staticmethod
    def similarity_matrix(lst_of_people, mode='dot_product'):
        matrix = np.zeros((len(people), len(people)))
        for i in range(len(people)):
            for j in range(len(people)):
                matrix[i][j] = people[i].similarity(people[j], mode=mode)
        return matrix


<h2>Method 1: Naive best-preference matching</h2>

Best-preference matching is pretty self-explanatory and intuitive, but let's see if it actually works!

In [488]:
people = [Participant(np.random.random(size=(3,))) for i in range(10)]
matrix1 = Participant.similarity_matrix(people, mode='dot_product')

## Matching people with the best preference isn't stable!
maxes1 = np.apply_along_axis(second_largest, 1, matrix1)

indices = []
for i in range(maxes1.shape[0]):
    indices.append(list(matrix1[i, :]).index(maxes1[i]))

print('Best preference matching:')
print(indices)

Best preference matching:
[2, 4, 0, 5, 7, 3, 3, 4, 9, 8]


There are 8 people that have unstable pairs. <b> This doesn't work!</b>

In [683]:
## Instead, let's rank each person's preferences using the similarity matrix.
def rank_indices(lst):
    _lst = list(lst)
    rankings = []
    sorted_lst = sorted(_lst, reverse=True)
    for i in sorted_lst:
        rankings.append(_lst.index(i))
    return np.array(rankings)

## The first column in rankings is useless because a person cannot rank themselves
rankings1 = np.apply_along_axis(rank_indices, 1, matrix1)[:, 1:]

print('\n')
print('matrix:')
print(matrix1)
print('\n')
print('rankings:')
print(rankings1)
## 



matrix:
[[1.         0.61194801 0.98135531 0.23939753 0.71064836 0.37769148
  0.32352403 0.81789351 0.86189651 0.81771022]
 [0.61194801 1.         0.46043098 0.40989402 0.97471511 0.50455052
  0.24338764 0.92032498 0.67367393 0.53748643]
 [0.98135531 0.46043098 1.         0.24394007 0.58920062 0.37345418
  0.37774432 0.72516531 0.84827799 0.83693463]
 [0.23939753 0.40989402 0.24394007 1.         0.55299687 0.98895001
  0.95795137 0.57506114 0.69854325 0.73461598]
 [0.71064836 0.97471511 0.58920062 0.55299687 1.         0.65088171
  0.43019003 0.98141146 0.81924534 0.71096742]
 [0.37769148 0.50455052 0.37345418 0.98895001 0.65088171 1.
  0.95331919 0.68312377 0.79503548 0.81786114]
 [0.32352403 0.24338764 0.37774432 0.95795137 0.43019003 0.95331919
  1.         0.50477183 0.7354148  0.80910319]
 [0.81789351 0.92032498 0.72516531 0.57506114 0.98141146 0.68312377
  0.50477183 1.         0.90791637 0.82106686]
 [0.86189651 0.67367393 0.84827799 0.69854325 0.81924534 0.79503548
  0.735414

<h2>Method 2: Exhaustive stable-marriage algorithm</h2>


Here we explore the pairing of n partcipants using the stable marriage algorithm. The algorithm stipulates that
the population have a "gender" feature for it to converge to a stable pairing(otherwise, some people would rather be with someone who has a higher priority on their wishlist). Source: http://www.eecs70.org/static/notes/n4.pdf

However, we don't have a natural gender feature in our use case. But what if we could just give someone a gender, for the sake of the problem? My solution to this problem is to exhaustively search every possible 50-50 gender "assignment" and find one that converges. 

Let's see if it works or not.

<h4>Step 1: Implementation of the stable marriage algorithm</h4>

In [690]:
class Male:
    def __init__(self, pref):
        self.pref = pref
        self.index = 0
    
    @property
    def askee(self):
        return self.pref[self.index]
        
class Female:
    def __init__(self, pref):
        self.pref = pref
        self.string = None

#########################
# This part should be automatically generated from the similarity matrix later
a = Male(['A', 'D', 'B', 'C'])
b = Male(['B', 'A', 'D', 'C'])
c = Male(['D', 'A', 'B', 'C'])
d = Male(['A', 'B', 'C', 'D'])
A = Female(['b', 'a', 'c', 'd'])
B = Female(['d', 'a', 'b', 'c'])
C = Female(['a', 'b', 'c', 'd'])
D = Female(['d', 'c', 'b', 'a'])

males = {'a': a, 'b': b, 'c': c, 'd': d}
females = {'A': A, 'B': B, 'C': C, 'D': D}

##########################

def stable_marriage(male_map, female_map):
    def ask():
        return {symbol: male.askee for symbol, male in male_map.items()}
    
    def groupby(dct):
        groups = {}
        for key, value in dct.items():
            try:
                groups[value].append(key)
            except KeyError:
                groups[value] = [key]
        return groups 

    def put_on_string(grouped):
        for woman_symbol, lst in grouped.items():
            try:
                current = female_map[woman_symbol]
                current.string = min(lst, key=lambda m: current.pref.index(m))
            except KeyError:
                raise

    def review_ask(askings):
        for man_symbol, woman_symbol in askings.items():
            if female_map[woman_symbol].string != man_symbol:
                male_map[man_symbol].index += 1
    
    count = 0
    while not all([female.string for female in female_map.values()]) and count < 100:
        try:
            askings = ask()
            grouped = groupby(askings)
            put_on_string(grouped)
            review_ask(askings)
            count += 1

        except KeyError as e:
            return 'There is no stable pairing' + str(e)
            
        
    return list(askings.items()) 
stable_marriage(males, females)

[('a', 'C'), ('b', 'A'), ('c', 'D'), ('d', 'B')]

<h4>Step 2: exhaustive search</h4>

In [734]:
from itertools import combinations

def split(source):
    c1 = combinations(source, len(source) // 2)
    def rest(source, combination):
        return tuple([el for el in source if el not in combination])

    return [(comb, rest(source, comb)) for comb in c1]


assignments = split(range(0, 10))


def remove_same_gender(part1, part2, rankings):
    cleaned_g1 = []
    cleaned_g2 = []
    g1 = rankings[list(part1)]
    g2 = rankings[list(part2)]
    diff_gender = lambda own_part: lambda x: x not in own_part
    for person in g1:
        cleaned_g1.append(np.apply_along_axis(lambda arr: list(filter(diff_gender(list(part1)), arr)), 0, person))
    for person in g2:
        cleaned_g2.append(np.apply_along_axis(lambda arr: list(filter(diff_gender(list(part2)), arr)), 0, person))
    return np.array(cleaned_g1), np.array(cleaned_g2)



## Do a smart repetition removal instead of making and freezeing sets. 
def without_reps(all_pairings):
    a = set()
    for pairing in all_pairings:
        s = set()
        for pair in pairing:
            s.add(frozenset(pair))
        a.add(frozenset(s))
    a = frozenset(a)
    return [list([set(pair) for pair in pairing]) for pairing in a]
        

## for demonstration purposes
def num_of_possible_pairs(m):
    def fact(n):
        if n == 0:
            return 1
        return n * fact(n-1)
    return fact(m) // (2**(m // 2) * fact(m // 2))
        
        
all_pairings = []
for part1, part2 in assignments:
    males, females = remove_same_gender(part1, part2, rankings1)
    male_map = dict(zip(part1, [Male(r.tolist()) for r in males]))
    female_map = dict(zip(part2, [Female(r.tolist()) for r in females]))
    all_pairings.append(stable_marriage(male_map, female_map))
    

optimal_pairings = without_reps(all_pairings)

ratio = len(optimal_pairings) / num_of_possible_pairs(10)
print(ratio)

0.03492063492063492


However, this only optimizes the stable marriage algorithm for different 1/2 population selections. It doesn't work.

<h2>Best algorithm: Stable Roommates Algorithm</h2>

Already implemented here: https://github.com/CoeCS/tacklebox/blob/master/stableroomate/stableroomate.py
This is python 2, and after a simple conversion it will be usable for us.